# 최적입지 선정 알고리즘

In [2]:
# 필요한 라이브러리 임포트&설치
# !pip install dbfread
# !pip install haversine
# !pip install sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from haversine import haversine
from tqdm.notebook import tqdm
%matplotlib inline
import platform
import pickle
import re
platform.system()
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'

지금 있는 데이터는 동별 라벨링이 모두 되어있으니, 초등학교 기준으로 나눈다고 치면, 한 초등학교가 선정되게 되면ㅇㅋㅇㅋ 써보자

In [4]:
def prepro(df):
#     df.reset_index(inplace=True)
    df_need = df.rename(columns=df.iloc[10]).drop(df.index[:11])
    return df_need

In [6]:
element_pop = pd.read_excel(r'./data/2019년+하반기+유초중등+교육통계조사+주요통계+세부자료.xlsx',sheet_name=4)
element_pop_need = prepro(element_pop)[['지역교육청','행정구역','학제','학교명','설립유형','학급수','학생수','교원']]
element_pop_need.dropna(inplace=True)
element_pop_df = element_pop_need[element_pop_need.학제=='초등학교']
element_df = element_pop_df[['행정구역','학교명','학생수']]
element_df

,행정구역,학교명,학생수
849,종로구,상명대학교사범대학부속초등학교,285
850,종로구,서울교동초등학교,174
851,종로구,서울대학교사범대학부설초등학교,622
852,종로구,서울독립문초등학교,495
853,종로구,서울매동초등학교,270
...,...,...,...
1451,강동구,서울위례초등학교,0
1452,강동구,서울천동초등학교,868
1453,강동구,서울천일초등학교,356
1454,강동구,서울천호초등학교,734


In [29]:
element_df['필요순위'] = 0
for i in range(element_df.shape[0]):
    element_df['필요순위'].iloc[i] = np.random.randint(0, (element_df.shape[0]))

<ipython-input-29-11bf0992821c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  element_df['필요순위'] = 0
C:\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-29-11bf0992821c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  element_df['필요순위'].iloc[i] = np.random.randint(0

In [36]:
element_df.iloc[1], element_df.iloc[2] = element_df.iloc[2],element_df.iloc[1]
element_df

b, c = a.iloc[0].copy(), a.iloc[1].copy()
a.iloc[0],a.iloc[1] = c,b

C:\anaconda3\lib\site-packages\pandas\core\indexing.py:1760: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


,행정구역,학교명,학생수,필요순위
849,종로구,상명대학교사범대학부속초등학교,285,546
850,종로구,서울매동초등학교,270,287
851,종로구,서울명신초등학교,389,381
852,종로구,서울세검정초등학교,857,364
853,종로구,서울재동초등학교,208,341
...,...,...,...,...
1451,강동구,서울천호초등학교,734,367
1452,강동구,서울한산초등학교,843,219
1453,강동구,서울강명초등학교,1320,409
1454,강동구,서울강일초등학교,1081,531


In [16]:
element_rank_df = element_df.sort_values('필요순위')
element_rank_df.head(20)

,행정구역,학교명,학생수,필요순위
1120,서대문구,서울대신초등학교,190,0
1169,양천구,서울신남초등학교,277,1
1380,강남구,서울역삼초등학교,335,2
1252,금천구,서울금산초등학교,339,4
983,성북구,서울동신초등학교,420,4
1035,도봉구,서울창도초등학교,925,5
946,동대문구,서울종암초등학교,570,5
1190,강서구,서울공항초등학교,1413,6
862,중구,동산초등학교,533,7
1161,양천구,서울계남초등학교,874,7


## 필수조건들:
#### 공간: 20평 이상-건축대장
#### 2021년 키움센터의 최종밀도 고려해서 마을단위 분산-haversine
#### 건물 노후화 확인-건축대장
#### 단열/에너지 확인-못함
#### 소음적은곳-못함
#### 화장실-못함


## 권장조건들
#### 거리: 학교 10분 이내-haversine
#### 1층/ 직접출입가능-건축대장
#### 놀이터/공원과 가까운곳-haversine
#### 아파트1층, 단독주택, 복지관-건축대장

In [21]:
element_df.iloc[1,:]

행정구역         종로구
학교명     서울매동초등학교
학생수          270
필요순위         544
Name: 850, dtype: object

In [38]:
element_rank_list = []
for i in range(element_rank_df.shape[0]):
    count = 0
    if count == 400:
        break
    if element_df.행정구역.iloc[i] == element_df.행정구역.iloc[i+1]:
        a, b = element_df.iloc[i+1].copy(), element_df.iloc[i+2].copy() 
        element_df.iloc[i+2], element_df.iloc[i+1] =a, b
    else:
        element_rank_list.append(element_df.학교명.iloc[i])
    count += 1
    
element_rank_list

IndexError: single positional indexer is out-of-bounds